In [321]:
import pandas as pd
import numpy as np

from mlgear.utils import show, display_columns
from itertools import chain


def clean_party(party):
    if not isinstance(party, str):
        return 'OTH'
    party = party.lower()
    if party.startswith('dem'):
        return 'DEM'
    elif party.startswith('rep'):
        return 'REP'
    elif party.startswith('ind'):
        return 'IND'
    else:
        return 'OTH'


def clean_election(election):
    if '-' in election:
        election = election.split('-')[0]
    if election == 'Sen':
        return 'senate'
    elif election == 'Pres':
        return 'president'
    elif election == 'Gov':
        return 'governor'
    else:
        return election


us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maine-1': 'M1',
    'Maine-2': 'M2',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nebraska-1': 'N1',
    'Nebraska-2': 'N2',
    'Nebraska-3': 'N3',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

def state_to_abbrev(state):
    return us_state_abbrev.get(state.title(), state.title())

def abbrev_to_state(abbrev):
    return abbrev_us_state.get(abbrev.upper(), abbrev.upper())


In [322]:
polls = pd.read_csv('data/polls_auxiliary_dataset.tsv', sep='\t')
polls2 = pd.read_csv('data/polls_main_dataset.tsv', sep='\t')
polls = pd.concat((polls, polls2)).reset_index(drop=True)

polls['populationType'] = polls['populationType'].fillna('LV')
polls = polls[polls['populationType'].isin(['LV', 'Likely Voters', 'lv'])]
polls = polls.drop('populationType', axis=1)

polls['dem_actual'] = polls['finalTwoPartyVSDemocratic']
polls['rep_actual'] = polls['finalTwoPartyVSRepublican']
polls['dem_polled'] = polls['democratic']
polls['rep_polled'] = polls['republican']
polls['undecided_polled'] = 100 - polls['dem_polled'] - polls['rep_polled']
polls['dem_diff'] = polls['dem_polled'] + polls['undecided_polled'] / 2 - polls['dem_actual']
polls['rep_diff'] = polls['rep_polled'] + polls['undecided_polled'] / 2 - polls['rep_actual']
polls['N'] = polls['numberOfRespondents']
polls['methodology'] = polls['Mode'].fillna('Unknown')
polls['pollster'] = polls['pollName']
polls['election'] = polls['election'].apply(clean_election)
polls['stage'] = 'general'
polls['population'] = 'lv'
polls['state'] = polls['state'].fillna('National').apply(abbrev_to_state)
polls['538grade'] = 'Unknown'
polls['days_until'] = (pd.to_datetime(polls['electionDate']) - pd.to_datetime(polls['startDate'])).dt.days

polls = polls[['election', 'state', 'year', 'stage', 'pollster', 'N', 'methodology', 'population',
               'days_until', '538grade', 'dem_polled', 'rep_polled', 'undecided_polled', 'dem_actual', 'rep_actual',
               'dem_diff', 'rep_diff']]
show(polls)

        election          state  year    stage  \
0      president        Alabama  2004  general   
1      president        Alabama  2004  general   
2      president        Alabama  2004  general   
3      president        Alabama  2004  general   
4      president        Alabama  2004  general   
...          ...            ...   ...      ...   
14660     senate  New Hampshire  2014  general   
14661     senate  New Hampshire  2014  general   
14662     senate  New Hampshire  2014  general   
14663     senate  New Hampshire  2014  general   
14664     senate  New Hampshire  2014  general   

                               pollster    N methodology population  \
0                            SurveyUSA   735     Unknown         lv   
1                       Capital Survey   645     Unknown         lv   
2                            SurveyUSA   699     Unknown         lv   
3                      Mobile Register   619     Unknown         lv   
4                                  ARG   699

In [323]:
polls_538_all = None

for p in ['senate', 'president', 'house', 'governor']:
    print('{}...'.format(p))
    polls_538 = pd.read_csv('data/{}_polls.csv'.format(p))
    polls_538['candidate'] = polls_538['candidate_name']
    polls_538['party'] = polls_538['candidate_party'].apply(clean_party)
    polls_538['poll_date'] = pd.to_datetime(polls_538['start_date'])
    polls_538['election_date'] = pd.to_datetime(polls_538['election_date'])
    polls_538['days_until'] = (polls_538['election_date'] - polls_538['poll_date']).dt.days
    polls_538['year'] = polls_538['cycle']
    polls_538['election'] = p
    polls_538['N'] = polls_538['sample_size']
    polls_538['538grade'] = polls_538['fte_grade'].fillna('Unknown')
    
    polls_538 = polls_538[['election', 'state', 'year', 'stage', 'pollster', 'N', 'methodology', 'population',
                           'days_until', '538grade', 'party', 'pct']]
    if polls_538_all is None:
        polls_538_all = polls_538
    else:
        polls_538_all = pd.concat((polls_538_all, polls_538)).reset_index(drop=True)
        
show(polls_538_all)

senate...
president...
house...
governor...
       election       state  year    stage                  pollster  \
0        senate     Georgia  2020   runoff                AtlasIntel   
1        senate     Georgia  2020   runoff                AtlasIntel   
2        senate     Georgia  2020   runoff                AtlasIntel   
3        senate     Georgia  2020   runoff                AtlasIntel   
4        senate     Georgia  2020   runoff           Trafalgar Group   
...         ...         ...   ...      ...                       ...   
25989  governor      Nevada  2018  general  Remington Research Group   
25990  governor  New Mexico  2018  general            Tarrance Group   
25991  governor  New Mexico  2018  general            Tarrance Group   
25992  governor    New York  2018  general     Public Policy Polling   
25993  governor    New York  2018  general     Public Policy Polling   

                N        methodology population  days_until 538grade party  \
0      857.00

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,22,25,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [324]:
polls_538_all['dem_polled'] = list(chain.from_iterable([[x, x] for x in polls_538_all['pct'].values[::2]]))
polls_538_all['rep_polled'] = list(chain.from_iterable([[x, x] for x in polls_538_all['pct'].values[1::2]]))
polls_538_all['undecided_polled'] = 100 - polls_538_all['dem_polled'] - polls_538_all['rep_polled']
polls_538_all = polls_538_all[polls_538_all['party'] == 'DEM']
polls_538_all = polls_538_all[['election', 'state', 'year', 'stage', 'pollster', 'N', 'methodology', 'population',
                               'days_until', '538grade', 'dem_polled', 'rep_polled', 'undecided_polled']]
show(polls_538_all)

       election       state  year    stage                        pollster  \
0        senate     Georgia  2020   runoff                      AtlasIntel   
2        senate     Georgia  2020   runoff                      AtlasIntel   
4        senate     Georgia  2020   runoff                 Trafalgar Group   
6        senate     Georgia  2020   runoff                 Trafalgar Group   
8        senate     Georgia  2020   runoff  Opinion Savvy/InsiderAdvantage   
...         ...         ...   ...      ...                             ...   
25984  governor        Ohio  2018  general                     Frank Luntz   
25986  governor     Arizona  2018  general           Public Policy Polling   
25988  governor      Nevada  2018  general        Remington Research Group   
25990  governor  New Mexico  2018  general                  Tarrance Group   
25992  governor    New York  2018  general           Public Policy Polling   

                N        methodology population  days_until 538

In [325]:
results = pd.read_csv('data/1976-2020-president.csv', encoding='ISO-8859-1')
results['party'] = results['party_simplified'].apply(clean_party)
results['election'] = 'president'
results['state'] = results['state'].apply(lambda s: s.title())
results['stage'] = 'general'
results['actual_pct'] = results['candidatevotes'] / results['totalvotes'] * 100
results = results[['election', 'state', 'year', 'stage', 'party', 'actual_pct']]

results_sen = pd.read_csv('data/1976-2020-senate.csv', encoding='ISO-8859-1')
results_sen['party'] = results_sen['party_simplified'].apply(clean_party)
results_sen['election'] = 'senate'
results_sen['state'] = results_sen['state'].apply(lambda s: s.title())
results_sen['stage'] = results_sen['stage'].apply(lambda x: 'general' if x == 'gen' else x)
results_sen['actual_pct'] = results_sen['candidatevotes'] / results_sen['totalvotes'] * 100
results_sen = results_sen[['election', 'state', 'year', 'stage', 'party', 'actual_pct']]

results = pd.concat((results, results_sen)).reset_index(drop=True)
results = results[results['party'] != 'OTH']
show(results)

results['dem_actual'] = list(chain.from_iterable([[x, x] for x in results['actual_pct'].values[::2]]))
results['rep_actual'] = list(chain.from_iterable([[x, x] for x in results['actual_pct'].values[1::2]]))
results = results[results['party'] == 'DEM']
results = results[['election', 'state', 'year', 'stage', 'dem_actual', 'rep_actual']]
show(results)

       election          state  year    stage party  actual_pct
0     president        Alabama  1976  general   DEM   55.727269
1     president        Alabama  1976  general   REP   42.614871
7     president         Alaska  1976  general   REP   57.904575
8     president         Alaska  1976  general   DEM   35.653131
11    president        Arizona  1976  general   REP   56.366136
...         ...            ...   ...      ...   ...         ...
7905     senate  West Virginia  2020  general   DEM   27.000146
7912     senate        Georgia  2021   runoff   REP   48.956147
7913     senate        Georgia  2021   runoff   DEM   51.043853
7914     senate        Georgia  2021   runoff   REP   49.383448
7915     senate        Georgia  2021   runoff   DEM   50.616552

[2840 rows x 6 columns]
(2840, 6)
       election          state  year    stage  dem_actual  rep_actual
0     president        Alabama  1976  general   55.727269   42.614871
8     president         Alaska  1976  general   57.904575

In [326]:
polls_538_all = polls_538_all.merge(results, on=['election', 'state', 'year', 'stage'], how='inner')
polls_538_all['undecided_polled'] = 100 - polls_538_all['dem_polled'] - polls_538_all['rep_polled']
polls_538_all['dem_diff'] = polls_538_all['dem_polled'] + polls_538_all['undecided_polled'] / 2 - polls_538_all['dem_actual']
polls_538_all['rep_diff'] = polls_538_all['rep_polled'] + polls_538_all['undecided_polled'] / 2 - polls_538_all['rep_actual']
show(polls_538_all)

       election   state  year    stage                        pollster  \
0        senate    Iowa  2020  general           Public Policy Polling   
1        senate    Iowa  2020  general                          Civiqs   
2        senate    Iowa  2020  general                 Change Research   
3        senate    Iowa  2020  general               Data for Progress   
4        senate    Iowa  2020  general                 Emerson College   
...         ...     ...   ...      ...                             ...   
8006  president  Alaska  2020  general  Zogby Interactive/JZ Analytics   
8007  president  Alaska  2020  general  Zogby Interactive/JZ Analytics   
8008  president  Alaska  2020  general  Zogby Interactive/JZ Analytics   
8009  president  Alaska  2020  general  Zogby Interactive/JZ Analytics   
8010  president  Alaska  2020  general  Zogby Interactive/JZ Analytics   

               N      methodology population  days_until 538grade  dem_polled  \
0     871.000000         IVR/T

In [327]:
ratings_538 = pd.read_csv('data/538pollster_ratings.csv')
show(ratings_538)

                                    Pollster  Pollster Rating ID  # of Polls  \
0                        Monmouth University                 215         108   
1                               Selzer & Co.                 304          48   
2               ABC News/The Washington Post                   3          73   
3    Siena College/The New York Times Upshot                 448          59   
4          Field Research Corp. (Field Poll)                  94          25   
..                                       ...                 ...         ...   
448                             SurveyMonkey                 324         210   
449                 Nielson Brothers Polling                 231           9   
450                             TCJ Research                 327         132   
451                       Humphrey Institute                 143          12   
452                       Jayhawk Consulting                 157           2   

    NCPP / AAPOR / Roper Live Caller Wi

In [328]:
raw_polls = pd.read_csv('data/538_raw_polls.csv')
raw_polls = raw_polls.loc[raw_polls['type_simple'].isin(['Pres-G', 'Sen-G', 'Gov-G', 'House-G'])]
raw_polls['state'] = raw_polls['location'].apply(abbrev_to_state)
raw_polls['election'] = raw_polls['type_simple'].apply(clean_election)
raw_polls['stage'] = 'general'
raw_polls['N'] = raw_polls['samplesize']
raw_polls = raw_polls.merge(ratings_538, left_on='pollster_rating_id', right_on='Pollster Rating ID')
raw_polls['methodology'] = raw_polls['Methodology']
raw_polls['population'] = 'lv'
raw_polls['538grade'] = raw_polls['538 Grade']
raw_polls['days_until'] = (pd.to_datetime(raw_polls['electiondate']) - pd.to_datetime(raw_polls['polldate'])).dt.days
raw_polls = raw_polls[raw_polls['cand1_party'] == 'DEM']
raw_polls['dem_polled'] = raw_polls['cand1_pct']
raw_polls['rep_polled'] = raw_polls['cand2_pct']
raw_polls['dem_actual'] = raw_polls['cand1_actual']
raw_polls['rep_actual'] = raw_polls['cand2_actual']
raw_polls['undecided_polled'] = 100 - raw_polls['dem_polled'] - raw_polls['rep_polled']
raw_polls['dem_diff'] = raw_polls['dem_polled'] + raw_polls['undecided_polled'] / 2 - raw_polls['dem_actual']
raw_polls['rep_diff'] = raw_polls['rep_polled'] + raw_polls['undecided_polled'] / 2 - raw_polls['rep_actual']
raw_polls = raw_polls[['election', 'state', 'year', 'stage', 'pollster', 'N', 'methodology',
                       'population', 'days_until', '538grade', 'dem_polled', 'rep_polled',
                       'undecided_polled', 'dem_actual', 'rep_actual', 'dem_diff', 'rep_diff']]
show(raw_polls)

      election        state  year    stage                           pollster  \
0     governor     New York  1998  general           Blum & Weprin Associates   
1       senate     New York  1998  general           Blum & Weprin Associates   
2     governor     New York  1998  general           Blum & Weprin Associates   
3     governor        Texas  1998  general           Blum & Weprin Associates   
4       senate     New York  1998  general           Blum & Weprin Associates   
...        ...          ...   ...      ...                                ...   
7714  governor    Louisiana  2019  general                  Data for Progress   
7715  governor    Louisiana  2019  general                    Spry Strategies   
7716  governor  Mississippi  2019  general                  Hickman Analytics   
7717  governor    Louisiana  2019  general  Edgewater Research/My People Vote   
7718  governor    Louisiana  2019  general  Edgewater Research/My People Vote   

               N      metho

In [329]:
polls_538_gcb = pd.read_csv('data/generic_ballot_polls.csv')
polls_538_gcb['poll_date'] = pd.to_datetime(polls_538_gcb['start_date'])
polls_538_gcb['election_date'] = pd.to_datetime('11/3/2020') # 2020
polls_538_gcb['year'] = polls_538_gcb['cycle']
polls_538_gcb.loc[polls_538_gcb['year'] == 2018, 'election_date'] = pd.to_datetime('11/6/2018')
polls_538_gcb['days_until'] = (polls_538_gcb['election_date'] - polls_538_gcb['poll_date']).dt.days
polls_538_gcb['election'] = 'GCB'
polls_538_gcb['state'] = 'National'
polls_538_gcb['N'] = polls_538_gcb['sample_size']
polls_538_gcb['stage'] = 'general'
polls_538_gcb['dem_polled'] = polls_538_gcb['dem']
polls_538_gcb['rep_polled'] = polls_538_gcb['rep']
polls_538_gcb['dem_actual'] = 50.8 # 2020 
polls_538_gcb['rep_actual'] = 47.7 # 2020
polls_538_gcb.loc[polls_538_gcb['year'] == 2018, 'dem_actual'] = 53.4
polls_538_gcb.loc[polls_538_gcb['year'] == 2018, 'rep_actual'] = 44.8
polls_538_gcb['undecided_polled'] = 100 - polls_538_gcb['dem_polled'] - polls_538_gcb['rep_polled']
polls_538_gcb['dem_diff'] = polls_538_gcb['dem_polled'] + polls_538_gcb['undecided_polled'] / 2 - polls_538_gcb['dem_actual']
polls_538_gcb['rep_diff'] = polls_538_gcb['rep_polled'] + polls_538_gcb['undecided_polled'] / 2 - polls_538_gcb['rep_actual']
polls_538_gcb['538grade'] = polls_538_gcb['fte_grade'].fillna('Unknown')
polls_538_gcb = polls_538_gcb[['election', 'state', 'year', 'stage', 'pollster', 'N', 'methodology', 'population',
                               'days_until', '538grade', 'dem_polled', 'rep_polled', 'undecided_polled',
                               'dem_actual', 'rep_actual', 'dem_diff', 'rep_diff']]
show(polls_538_gcb)

     election     state  year    stage  \
0         GCB  National  2020  general   
1         GCB  National  2020  general   
2         GCB  National  2020  general   
3         GCB  National  2020  general   
4         GCB  National  2020  general   
...       ...       ...   ...      ...   
2701      GCB  National  2018  general   
2702      GCB  National  2018  general   
2703      GCB  National  2018  general   
2704      GCB  National  2018  general   
2705      GCB  National  2018  general   

                                          pollster           N methodology  \
0                                           YouGov 1500.000000      Online   
1                                           YouGov 1363.000000      Online   
2     John Zogby Strategies/EMI Research Solutions 1008.000000      Online   
3                                           YouGov 1360.000000      Online   
4                                           YouGov 1501.000000      Online   
...                        

In [330]:
polls = pd.concat((polls, polls_538_all, polls_538_gcb, raw_polls)).reset_index(drop=True).drop_duplicates()
show(polls)

        election        state  year    stage  \
0      president      Alabama  2004  general   
1      president      Alabama  2004  general   
2      president      Alabama  2004  general   
3      president      Alabama  2004  general   
4      president      Alabama  2004  general   
...          ...          ...   ...      ...   
31224   governor    Louisiana  2019  general   
31225   governor    Louisiana  2019  general   
31226   governor  Mississippi  2019  general   
31227   governor    Louisiana  2019  general   
31228   governor    Louisiana  2019  general   

                                pollster           N      methodology  \
0                             SurveyUSA   735.000000          Unknown   
1                        Capital Survey   645.000000          Unknown   
2                             SurveyUSA   699.000000          Unknown   
3                       Mobile Register   619.000000          Unknown   
4                                   ARG   699.000000      

In [331]:
display_columns(polls)

## 538grade ##
Unknown    43.026939
B/C        11.431837
D-          9.090612
B-          5.903673
B           5.201633
C+          3.454694
B+          3.350204
C           3.294694
A           3.036735
A+          2.971429
A/B         2.589388
A-          2.174694
F           1.720816
C-          1.495510
C/D         1.093878
D+          0.163265
Name: 538grade, dtype: float64
-
-
## N ##
Mean: 1200.1930509286476 Median: 724.0 SD: 2067.151480592336
-
-
## days_until ##
Mean: 90.93010530662812 Median: 21.0 SD: 141.24343409338374
-
-
## dem_actual ##
Mean: 49.58242247971995 Median: 50.8 SD: 10.098082309442015
-
-
## dem_diff ##
Mean: 0.7033326223208647 Median: 0.1765799300000026 SD: 9.357271178645592
-
-
## dem_polled ##
Mean: 45.062594612244894 Median: 46.0 SD: 9.09211608607582
-
-
## election ##
president    42.540408
senate       30.739592
governor     13.175510
GCB           8.829388
House         4.715102
Name: election, dtype: float64
-
-
## methodology ##
Unknown                

In [333]:
polls.to_csv('polls.csv', index=False)